In [722]:
# Importing the modules
import pandas as pd
import numpy as np
import openpyxl

import os
import sys
from bs4 import BeautifulSoup

from IPython.display import display

### Loading and displaying data / データの読み込みと表示

In [723]:
#look at raw data and plan cleaning

df_A = pd.read_excel('MasterDuel_1-20.xlsx', sheet_name='Sheet1')

#creating function to display tables in advance
def show_df_A(n):
    print("\n df_A:\n")
    display(df_A.head(n))

show_df_A(2)

df_B = pd.read_csv('MasterDuel_21-50.csv')

#creating function to display tables in advance
def show_df_B(n):
    print("\n df_B:\n")
    display(df_B.head(n))

show_df_B(5)


 df_A:



,No.,Results,Deck name,Date,Datetime(start),Datetime(end),All turn,First Turn,First Card,Second Card,Third Card,Fourth Card,Fifth card,Sixth card,N of Handtrap,N of monsters,Opponents deck name,N of Opponents handtrap,Rank,Tier
0,1,Win,valet without maid,2022-03-02 00:00:00,NaN,23:14:00,NaN,1,セイファート,コラブサーペント,ヴァレット・トレーサー,龍の鏡,マスク・チェンジ・セカンド,NaN,NaN,NaN,閃刀姫,NaN,G,4
1,2,Win,valet without maid,2022-03-02 00:00:00,23:28:00,00:02:00,NaN,0,原始生命体ニビル,暗黒竜コラプサーペント,カオス・テリトリー,星遺物の守護竜,サイフレームギア・ドライバー,クイック・リボルブ,NaN,NaN,閃刀姫,"増殖するG, 灰流うらら\n灰流うらら",G,4



 df_B:



,No.,Rank,Result,Datetime(start),Datetime(end),Sum_turn,First_turn,First_card,Second_card,Third_card,Fourth_card,Fifth_card,Opp_decktype,Opp_handtrap,Opp's handtrap,Property
0,21,G2,Win,"@March 9, 2022","@March 6, 2022 3:00 AM",NaN,Turn2,PSYフレームギア・γ,墓穴の指名者,ブレイクスルー・スキル,サンダー・ボルト,ヴァレット・シンクロン,転生炎獣,NaN,NaN,NaN
1,22,G2,Win,"@March 6, 2022","@March 6, 2022 3:08 AM",2.0,Turn1,増殖するG,PSYフレーム・ドライバー,増殖するG,暗黒竜 コラプサーペント,クイック・リボルブ,ブルーアイズ,0.0,NaN,NaN
2,30,G2,Win,"@March 8, 2022 11:45 PM","@March 8, 2022 12:00 AM",2.0,Turn2,リボルブート・セクター,輝光竜セイファート,王宮の勅命,ヴァレット・リチャージャー,ヴァレット・トレーサー,絵札の三銃士,0.0,NaN,NaN
3,31,G2,Win,"@March 9, 2022 12:20 AM","@March 9, 2022 12:44 AM",3.0,Turn1,リボルブート・セクター,ヴァレット・トレーサー,混沌領域,王宮の勅命,メタルヴァレット・ドラゴン,ギャラクシー,0.0,NaN,NaN
4,32,G2,Win,"@March 9, 2022 12:43 AM","@March 9, 2022 1:07 AM",7.0,Turn1,クイック・リボルブ,メタルヴァレット・ドラゴン,灰流うらら,ヴァレット・トレーサー,ヴァレット・トレーサー,罠バーン,0.0,NaN,NaN


### Plan how to clean / データの整形方法を計画

* Issues / 課題:
  - Unimportant rows or columns / 重要でない行や列:
  - Naming and merging columns correctly / カラムの命名と結合を正しく行う :
    - Some column name is not consistent / 一部のカラム名が統一されていない
    - Date and timestamp value needs to be merged / 日付とタイムスタンプの値を統合する必要
  - Inconsistent data formats　/ 一貫性のないデータフォーマット :
    - Null data, Some records have missing value / 一部のレコードに値がない
    - Datetime values are broken /  Datetimeの値が壊れています

* Data cleaning steps / データクリーニングの手順:

1. Decide which columns to use and define table columns. / 使用する列を決定し、テーブルの列を定義する。

2. Delete unnecessary columns. / 不要な列を削除する。

3. Merge, separate or rename columns. / 列を結合したり、分離したり、名前を変えたり

   Format value. / データをフォーマット化する。

4. Combine two dataframe into one. / ふたつのデータフレームをひとつに。

### 1. Decide which columns to use and define table columns. / 使用する列を決定し、テーブルの列を定義する。

In [724]:
#define function to use it later 
def compare_columns(A,B):
    A_col = A.columns.values.tolist()
    B_col = B.columns.values.tolist()

    #show unmatched column between two tables
    diff = set(A_col) ^ set(B_col)
    print("\n Unmatched column name:\n")
    display(diff)

    #show which column has a common name with the other table.
    common = set(A_col) & set(B_col)
    print("\n Matched column name：{}\n".format(common))

    #Line up column name
    s1 = pd.Series(A_col, name='A')
    s2 = pd.Series(B_col, name='B')
    compare_col = pd.concat([s1, s2], axis=1)
    print("\n compare column name:\n")
    display(compare_col)

compare_columns(df_A, df_B)


 Unmatched column name:



{'All turn',
 'Date',
 'Deck name',
 'Fifth card',
 'Fifth_card',
 'First Card',
 'First Turn',
 'First_card',
 'First_turn',
 'Fourth Card',
 'Fourth_card',
 'N of Handtrap',
 'N of Opponents handtrap',
 'N of monsters',
 "Opp's handtrap",
 'Opp_decktype',
 'Opp_handtrap',
 'Opponents deck name',
 'Property',
 'Result',
 'Results',
 'Second Card',
 'Second_card',
 'Sixth card',
 'Sum_turn',
 'Third Card',
 'Third_card',
 'Tier'}


 Matched column name：{'Rank', 'No.', 'Datetime(end)', 'Datetime(start)'}


 compare column name:



,A,B
0,No.,No.
1,Results,Rank
2,Deck name,Result
3,Date,Datetime(start)
4,Datetime(start),Datetime(end)
5,Datetime(end),Sum_turn
6,All turn,First_turn
7,First Turn,First_card
8,First Card,Second_card
9,Second Card,Third_card


### 2. Delete unnecessary columns / 不要な列を削除する

In [725]:
# delete multiple colunms
df_A = df_A.drop(['Deck name','All turn','Sixth card','N of Handtrap','N of monsters'], axis=1)
df_B = df_B.drop('Sum_turn', axis=1)

### 3. Merge, separate or rename columns. / 列をマージしたり、分離したり。
###    Format value. / データをフォーマット化する。

##### Start from datetime related columns. / Datetimeと関連した列から始める。

In [726]:
print("\n df_A:\n")
display(df_A[['Date', 'Datetime(start)','Datetime(end)']].head(3))
print("\n df_B:\n")
display(df_B[['Datetime(start)', 'Datetime(end)']].head(3))


 df_A:



,Date,Datetime(start),Datetime(end)
0,2022-03-02 00:00:00,NaN,23:14:00
1,2022-03-02 00:00:00,23:28:00,00:02:00
2,2022-03-02 00:00:00,00:03:00,00:08:00



 df_B:



,Datetime(start),Datetime(end)
0,"@March 9, 2022","@March 6, 2022 3:00 AM"
1,"@March 6, 2022","@March 6, 2022 3:08 AM"
2,"@March 8, 2022 11:45 PM","@March 8, 2022 12:00 AM"


Since I found that the data contained empty string, I had to turn those to NaN value.
<- Otherwise, the functions won't work

空文字列が含まれていることがわかったので、それをNaN値にしておく必要がある。
<-そうしないと以降の、関数が動作しない。

In [727]:
# turn empty string to NaN
df_A = df_A.replace(r'^\s*$', np.nan, regex=True)
df_B = df_B.replace(r'^\s*$', np.nan, regex=True)
show_df_B(6)


 df_B:



,No.,Rank,Result,Datetime(start),Datetime(end),First_turn,First_card,Second_card,Third_card,Fourth_card,Fifth_card,Opp_decktype,Opp_handtrap,Opp's handtrap,Property
0,21,G2,Win,"@March 9, 2022","@March 6, 2022 3:00 AM",Turn2,PSYフレームギア・γ,墓穴の指名者,ブレイクスルー・スキル,サンダー・ボルト,ヴァレット・シンクロン,転生炎獣,NaN,NaN,NaN
1,22,G2,Win,"@March 6, 2022","@March 6, 2022 3:08 AM",Turn1,増殖するG,PSYフレーム・ドライバー,増殖するG,暗黒竜 コラプサーペント,クイック・リボルブ,ブルーアイズ,0.0,NaN,NaN
2,30,G2,Win,"@March 8, 2022 11:45 PM","@March 8, 2022 12:00 AM",Turn2,リボルブート・セクター,輝光竜セイファート,王宮の勅命,ヴァレット・リチャージャー,ヴァレット・トレーサー,絵札の三銃士,0.0,NaN,NaN
3,31,G2,Win,"@March 9, 2022 12:20 AM","@March 9, 2022 12:44 AM",Turn1,リボルブート・セクター,ヴァレット・トレーサー,混沌領域,王宮の勅命,メタルヴァレット・ドラゴン,ギャラクシー,0.0,NaN,NaN
4,32,G2,Win,"@March 9, 2022 12:43 AM","@March 9, 2022 1:07 AM",Turn1,クイック・リボルブ,メタルヴァレット・ドラゴン,灰流うらら,ヴァレット・トレーサー,ヴァレット・トレーサー,罠バーン,0.0,NaN,NaN
5,33,G2,Win,"@March 9, 2022 1:09 AM",NaN,Turn2,灰流うらら,輝白竜 ワイバースター,PSYフレーム・ドライバー,リボルブート・セクター,墓穴の指名者,ブラック・マジシャン,0.0,NaN,NaN


For df_A:

Column name: Date

-> leave out h, min, sec. / 時、分、秒を除外する。

Column name: Datetime(start), Datetime(end)

-> Format value to dateime. / 値をdatetimeにフォーマットする 

For df_B:

Column name: Datetime(start), Datetime(end)

-> Take out mm,dd, and then turn it to column 'date'/ mm,ddを取り出し、列「date」にする。

-> Format value to dateime / 値をdatetimeにフォーマットする 

* df_A['Date']

In [728]:
df_A['Date']

0     2022-03-02 00:00:00
1     2022-03-02 00:00:00
2     2022-03-02 00:00:00
3                   44623
4                   44623
5                   44623
6                   44625
7                   44625
8                   44625
9     2022-03-05 00:00:00
10    2022-03-05 00:00:00
11    2022-03-05 00:00:00
12    2022-03-05 00:00:00
13    2022-03-05 00:00:00
14    2022-03-05 00:00:00
15    2022-03-06 00:00:00
16    2022-03-06 00:00:00
17    2022-03-06 00:00:00
18    2022-03-06 00:00:00
19    2022-03-06 00:00:00
Name: Date, dtype: object

In [729]:
from datetime import datetime, timedelta

m = df_A['Date'].astype(str).str.isdigit()

def from_excel_ordinal(ordinal: float, _epoch0=datetime(1899, 12, 31)) -> datetime:
    if ordinal >= 60:
        ordinal -= 1  # Excel leap year bug, 1900 is not a leap year!
    return (_epoch0 + timedelta(days=ordinal)).replace(microsecond=0)

df_A.loc[m, 'Date'] = \
df_A.loc[m, 'Date']\
  .astype(int)\
  .apply(from_excel_ordinal)

df_A['Date'] = pd.to_datetime(df_A['Date'], errors='coerce').dt.strftime('%B %d, %Y') #Using errors='coerce'. It will replace all non-numeric values with NaN. 
display(df_A['Date'])

0     March 02, 2022
1     March 02, 2022
2     March 02, 2022
3     March 03, 2022
4     March 03, 2022
5     March 03, 2022
6     March 05, 2022
7     March 05, 2022
8     March 05, 2022
9     March 05, 2022
10    March 05, 2022
11    March 05, 2022
12    March 05, 2022
13    March 05, 2022
14    March 05, 2022
15    March 06, 2022
16    March 06, 2022
17    March 06, 2022
18    March 06, 2022
19    March 06, 2022
Name: Date, dtype: object

* df_A['Datetime(start)'], df_A['Datetime(end)']

In [730]:
df_A[['Datetime(start)','Datetime(end)']]

,Datetime(start),Datetime(end)
0,NaN,23:14:00
1,23:28:00,00:02:00
2,00:03:00,00:08:00
3,11:00:00,11:15:00
4,11:15:00,11:25:00
5,11:25:00,11:30:00
6,NaN,NaN
7,NaN,NaN
8,02:02:00,02:22:00
9,02:22:00,NaN


In [731]:
df_A = df_A.rename(columns={
            'Datetime(start)':"Time(start)",
            'Datetime(end)':"Time(end)"
            })

show_df_A(2)


 df_A:



,No.,Results,Date,Time(start),Time(end),First Turn,First Card,Second Card,Third Card,Fourth Card,Fifth card,Opponents deck name,N of Opponents handtrap,Rank,Tier
0,1,Win,"March 02, 2022",NaN,23:14:00,1,セイファート,コラブサーペント,ヴァレット・トレーサー,龍の鏡,マスク・チェンジ・セカンド,閃刀姫,NaN,G,4
1,2,Win,"March 02, 2022",23:28:00,00:02:00,0,原始生命体ニビル,暗黒竜コラプサーペント,カオス・テリトリー,星遺物の守護竜,サイフレームギア・ドライバー,閃刀姫,"増殖するG, 灰流うらら\n灰流うらら",G,4


df_B:

In [732]:
display(df_B[['Datetime(start)', 'Datetime(end)']].head(3))

,Datetime(start),Datetime(end)
0,"@March 9, 2022","@March 6, 2022 3:00 AM"
1,"@March 6, 2022","@March 6, 2022 3:08 AM"
2,"@March 8, 2022 11:45 PM","@March 8, 2022 12:00 AM"


Column name: Datetime(start), Datetime(end)

->Check if each value follows same format　/ 各値が同じ書式かどうか確認する

->Insert slash in each value with a certain position / 各値の特定の位置にスラッシュを挿入する

->Separate values with a slash into df_B['Date'] and df_A['Time(start)'] / スラッシュで区切られた値をdf_B['Date']とdf_A['Time(start)']に分割する

->Format value to datetime / 値を datetime にフォーマットする。

In [733]:
# check if each value follows same format
df_B['Count_spaces'] = df_B['Datetime(start)'].astype(str).str.count('\s+')
print("\ndf_B:\n")
display(df_B[['Datetime(start)','Count_spaces']].head())
df_B = df_B.drop(['Count_spaces'], axis=1)


df_B:



,Datetime(start),Count_spaces
0,"@March 9, 2022",2
1,"@March 6, 2022",2
2,"@March 8, 2022 11:45 PM",4
3,"@March 9, 2022 12:20 AM",4
4,"@March 9, 2022 12:43 AM",4


-> We can divide values to follow two patterns; / 2つのパターンに従って値を分割することができる。;

A: one with date only, / A: 日付のみのもの。

B: one with date and time. / B: 日付と時刻を含むもの。

The key to determining which pattern one record follow
is whether a value has AM/PM letter.
/ どちらのパターンで記録するかを決めるポイントは
AM/PMの文字があるかどうか。

In [734]:
df_B.iloc[5,4]

nan

In [735]:
#inserting slash in each value with a certain position
Key_word = ("AM","PM")
df_B['Datetime(start)'] = df_B['Datetime(start)'].apply(lambda x: x[1:-8] + "/" + x[-7:]  if (pd.notnull(x) and x.endswith(Key_word)) else (x[1:17] + "/" if pd.notnull(x) else x))
df_B['Datetime(end)'] = df_B['Datetime(end)'].apply(lambda x: x[1:-8] + "/" + x[-7:]  if (pd.notnull(x) and x.endswith(Key_word)) else (x[1:17] + "/" if pd.notnull(x) else x))

print("\n df_B:\n")
display(df_B[['Datetime(start)','Datetime(end)']].head(6))

#error log: below codes could not deal with misssing value
#pd.to_datetime(df_A['Date'] + ' ' + df_A['Datetime(start)'])
#str(df_A['Date']) + ' ' + str(df_A['Datetime(start)'])
# pd.to_datetime(str(df_A['Date']) + ' ' + str(df_A['Datetime(start)']))



 df_B:



,Datetime(start),Datetime(end)
0,"March 9, 2022/","March 6, 2022/3:00 AM"
1,"March 6, 2022/","March 6, 2022/3:08 AM"
2,"March 8, 2022 /1:45 PM","March 8, 2022 /2:00 AM"
3,"March 9, 2022 /2:20 AM","March 9, 2022 /2:44 AM"
4,"March 9, 2022 /2:43 AM","March 9, 2022/1:07 AM"
5,"March 9, 2022/1:09 AM",NaN


In [736]:
# Separate values with a slash into df_B['Date'] and df_A['Time(start)']
df_B[['Date','Datetime(start)']] = df_B['Datetime(start)'].str.split('/', expand=True)
df_B['Datetime(end)'] = df_B['Datetime(end)'].apply(lambda x: x[-7:] if pd.notnull(x) else x) #notnull helps to skip null value

df_B = df_B.rename(columns={
            'Datetime(start)':"Time(start)",
            'Datetime(end)':"Time(end)"
            })

# display the dataframe
print("\n df_B:\n")
display(df_B[['Date','Time(start)','Time(end)']].head(6))


 df_B:



,Date,Time(start),Time(end)
0,"March 9, 2022",,3:00 AM
1,"March 6, 2022",,3:08 AM
2,"March 8, 2022",1:45 PM,2:00 AM
3,"March 9, 2022",2:20 AM,2:44 AM
4,"March 9, 2022",2:43 AM,1:07 AM
5,"March 9, 2022",1:09 AM,NaN


Convert value to datetime

In [737]:
#Format value to datetime
df_B['Date'] = pd.to_datetime(df_B['Date']).dt.strftime('%B %d, %Y')
df_B['Time(start)'] = pd.to_datetime(df_B['Time(start)']).dt.strftime('%H:%M:%S')
df_B['Time(end)'] = pd.to_datetime(df_B['Time(end)']).dt.strftime('%H:%M:%S')

#Error log: ParserError: String does not contain a date: /
#all values had to be in datetime object
#-> Leaning 2 

In [738]:
display(df_B[['Date','Time(start)','Time(end)']].head(6))
df_B.info()

,Date,Time(start),Time(end)
0,"March 09, 2022",NaN,03:00:00
1,"March 06, 2022",NaN,03:08:00
2,"March 08, 2022",13:45:00,02:00:00
3,"March 09, 2022",02:20:00,02:44:00
4,"March 09, 2022",02:43:00,01:07:00
5,"March 09, 2022",01:09:00,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   No.             30 non-null     object 
 1   Rank            29 non-null     object 
 2   Result          30 non-null     object 
 3   Time(start)     24 non-null     object 
 4   Time(end)       29 non-null     object 
 5   First_turn      30 non-null     object 
 6   First_card      30 non-null     object 
 7   Second_card     30 non-null     object 
 8   Third_card      30 non-null     object 
 9   Fourth_card     30 non-null     object 
 10  Fifth_card      30 non-null     object 
 11  Opp_decktype    30 non-null     object 
 12  Opp_handtrap    17 non-null     float64
 13  Opp's handtrap  7 non-null      object 
 14  Property        10 non-null     object 
 15  Date            27 non-null     object 
dtypes: float64(1), object(15)
memory usage: 3.9+ KB


##### Deal with columns that are not related with datetime / Datetime以外の列を編集する

Column name: df_A['Rank'] , df_A['Tier']

-> Concatenate them / 2つのコラムの値を連結

In [739]:
# Concatenate two columns
df_A['Rank'] = df_A['Rank'] + "" + df_A['Tier'].astype(str)
df_A = df_A.drop('Tier', axis=1)
df_A.head(2)

,No.,Results,Date,Time(start),Time(end),First Turn,First Card,Second Card,Third Card,Fourth Card,Fifth card,Opponents deck name,N of Opponents handtrap,Rank
0,1,Win,"March 02, 2022",NaN,23:14:00,1,セイファート,コラブサーペント,ヴァレット・トレーサー,龍の鏡,マスク・チェンジ・セカンド,閃刀姫,NaN,G4
1,2,Win,"March 02, 2022",23:28:00,00:02:00,0,原始生命体ニビル,暗黒竜コラプサーペント,カオス・テリトリー,星遺物の守護竜,サイフレームギア・ドライバー,閃刀姫,"増殖するG, 灰流うらら\n灰流うらら",G4


Rename columns name / カラム名の変更

In [740]:
# Rename other column name
df_A = df_A.rename(columns={
            'Results':'Result',
            'First Turn':'First_turn',
            'First Card':'First_card',
            'Second Card':'Second_card',
            'Third Card':'Third_card',
            'Fourth Card':'Fourth_card',
            'Fifth card':'Fifth_card', # be careful with capitalization
            'Opponents deck name':'Opp_decktype',
            'N of Opponents handtrap':'Name_of_Opp_handtrap'
            })

df_B = df_B.rename(columns={
            'Opp_handtrap':'N_of_Opp_handtrap',
            "Opp's handtrap":"Name_of_Opp_handtrap"})

In [741]:
compare_columns(df_A, df_B)



 Unmatched column name:



{'N_of_Opp_handtrap', 'Property'}


 Matched column name：{'First_card', 'No.', 'Time(end)', 'Third_card', 'Result', 'Fourth_card', 'Rank', 'First_turn', 'Name_of_Opp_handtrap', 'Time(start)', 'Opp_decktype', 'Fifth_card', 'Date', 'Second_card'}


 compare column name:



,A,B
0,No.,No.
1,Result,Rank
2,Date,Result
3,Time(start),Time(start)
4,Time(end),Time(end)
5,First_turn,First_turn
6,First_card,First_card
7,Second_card,Second_card
8,Third_card,Third_card
9,Fourth_card,Fourth_card


### 4. Combine two dataframe into one  / ふたつのデータフレームをひとつに

In [742]:
df_combine = pd.concat([df_A, df_B]).set_index('No.')
df_combine = df_combine.reindex(columns=['Result', 'Date', 'Time(start)', 'Time(end)', 'Rank','First_turn', 'First_card', 'Second_card', 'Third_card', 'Fourth_card', 'Fifth_card', 'Opp_decktype', 'N_of_Opp_handtrap', 'Name_of_Opp_handtrap', 'Property'])
print(df_combine.columns.values.tolist())
print("\n df_combine:\n")
display(df_combine.iloc[18:22,:])
df_combine.info()

['Result', 'Date', 'Time(start)', 'Time(end)', 'Rank', 'First_turn', 'First_card', 'Second_card', 'Third_card', 'Fourth_card', 'Fifth_card', 'Opp_decktype', 'N_of_Opp_handtrap', 'Name_of_Opp_handtrap', 'Property']

 df_combine:



,Result,Date,Time(start),Time(end),Rank,First_turn,First_card,Second_card,Third_card,Fourth_card,Fifth_card,Opp_decktype,N_of_Opp_handtrap,Name_of_Opp_handtrap,Property
No.,,,,,,,,,,,,,,,
19,Win,"March 06, 2022",NaN,02:29:00,G3,0,ヴァレット・シンクロン,マグナヴァレット・ドラゴン,メタルヴァレット・ドラゴン,龍の鏡,クイック・リボルブ,ブルーアイズ,NaN,NaN,NaN
20,Lose,"March 06, 2022",NaN,02:37:00,G3,0,ヴァレット・トレーサー,ハーピィの羽箒,竜の渓谷,クイック・リボルブ,クイック・リボルブ,怪獣×除外系,NaN,ディメンションアトラクター,NaN
21,Win,"March 09, 2022",NaN,03:00:00,G2,Turn2,PSYフレームギア・γ,墓穴の指名者,ブレイクスルー・スキル,サンダー・ボルト,ヴァレット・シンクロン,転生炎獣,NaN,NaN,NaN
22,Win,"March 06, 2022",NaN,03:08:00,G2,Turn1,増殖するG,PSYフレーム・ドライバー,増殖するG,暗黒竜 コラプサーペント,クイック・リボルブ,ブルーアイズ,0.0,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 1 to 29
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Result                50 non-null     object 
 1   Date                  47 non-null     object 
 2   Time(start)           35 non-null     object 
 3   Time(end)             44 non-null     object 
 4   Rank                  49 non-null     object 
 5   First_turn            50 non-null     object 
 6   First_card            50 non-null     object 
 7   Second_card           50 non-null     object 
 8   Third_card            49 non-null     object 
 9   Fourth_card           50 non-null     object 
 10  Fifth_card            50 non-null     object 
 11  Opp_decktype          49 non-null     object 
 12  N_of_Opp_handtrap     17 non-null     float64
 13  Name_of_Opp_handtrap  14 non-null     object 
 14  Property              10 non-null     object 
dtypes: float64(1), object(14)
memo

### Format value with consistency. / 値を整える.

Before formatting the value, I had to fix the index at 50, which was in Japanese letters. / 値をフォーマットする前に、日本語の文字で５０となっていたインデックスを50にする必要があった.

In [743]:
#issue
display(df_combine.iloc[41:43,:])

,Result,Date,Time(start),Time(end),Rank,First_turn,First_card,Second_card,Third_card,Fourth_card,Fifth_card,Opp_decktype,N_of_Opp_handtrap,Name_of_Opp_handtrap,Property
No.,,,,,,,,,,,,,,,
49,Win,"March 12, 2022",13:57:00,14:11:00,G2,Turn2,サンダー・ボルト,墓穴の指名者,輝白竜 ワイバースター,アブソルーター・ドラゴン,マスク・チェンジ・セカンド,罠メタ,NaN,NaN,ハーピィの羽根帚を引いから罠を破壊できた
５０,Lose,"March 12, 2022",14:17:00,14:47:00,G2,Turn1,クイック・リボルブ,クイック・リボルブ,輝白竜 ワイバースター,暗黒竜 コラプサーペント,龍の鏡,竜儀巧,NaN,NaN,NaN


In [744]:
#solution
df_combine = df_combine.rename(index={'５０':50})
df_combine.index = df_combine.index.astype(int)
df_combine.sort_index(ascending=True, inplace=True)

Change value at a certain position to the desired format / ある位置の値を任意の形式に変更する 

In [745]:
#issue
df_combine.iloc[18:22,5]

No.
19        0
20        0
21    Turn2
22    Turn1
Name: First_turn, dtype: object

In [746]:
#format value by applying lambda function
df_combine.iloc[0:20,5] = df_combine.iloc[0:20,5].apply(lambda x : x if x == 1 else 2)
df_combine.iloc[20:50,5] = df_combine.iloc[20:50,5].apply(lambda x : 1 if x == "Turn1" else ( 2 if x == "Turn2" else np.nan))
df_combine.iloc[18:22,5:6]

,First_turn
No.,
19,2
20,2
21,2
22,1


Count how many card names appeared in a value of 'Name_of_Opp_handtrap' and turn it to the value of 'N_of_Opp_handtrap'. / 'Name_of_Opp_handtrap'の値に何枚のカード名が登場したかを数え、'N_of_Opp_handtrap'の値とする。

In [747]:
#issue
df_combine.iloc[[19,23],12:14]

,N_of_Opp_handtrap,Name_of_Opp_handtrap
No.,,
20,NaN,ディメンションアトラクター
24,NaN,NaN


In [748]:
#soulution
df_combine['Name_of_Opp_handtrap'] = df_combine['Name_of_Opp_handtrap'].str.replace('\n',', ')
df_combine['Name_of_Opp_handtrap'] = df_combine['Name_of_Opp_handtrap'].fillna("")
df_combine['N_of_Opp_handtrap'] = df_combine['Name_of_Opp_handtrap'].apply(lambda x: len(x.split(",")) if not x == "" else 0)
df_combine.iloc[[19,23],12:14]

,N_of_Opp_handtrap,Name_of_Opp_handtrap
No.,,
20,1,ディメンションアトラクター
24,0,


### Finished cleaning!　/  データクリーニング完！

In [749]:
df_combine

,Result,Date,Time(start),Time(end),Rank,First_turn,First_card,Second_card,Third_card,Fourth_card,Fifth_card,Opp_decktype,N_of_Opp_handtrap,Name_of_Opp_handtrap,Property
No.,,,,,,,,,,,,,,,
1,Win,"March 02, 2022",NaN,23:14:00,G4,1,セイファート,コラブサーペント,ヴァレット・トレーサー,龍の鏡,マスク・チェンジ・セカンド,閃刀姫,0,,NaN
2,Win,"March 02, 2022",23:28:00,00:02:00,G4,2,原始生命体ニビル,暗黒竜コラプサーペント,カオス・テリトリー,星遺物の守護竜,サイフレームギア・ドライバー,閃刀姫,3,"増殖するG, 灰流うらら, 灰流うらら",NaN
3,Lose,"March 02, 2022",00:03:00,00:08:00,G4,2,増殖するG,輝白竜ワイバースター,ハーピィの羽箒,ドラゴンズ・ミラー,クイック・リボルブ,トライブリゲート,0,,NaN
4,Win,"March 03, 2022",11:00:00,11:15:00,G4,2,はるうらら,アブソルーター・ドラゴン,竜の渓谷,星遺物の守護竜,カオス・テリトリー,蟲惑魔,0,,NaN
5,Lose,"March 03, 2022",11:15:00,11:25:00,G4,1,増殖するG,サンダーボルト,竜の渓谷,マスク・チェンジ・セカンド,墓穴の指名者,ヌメロン,1,増殖するG,NaN
6,Lose,"March 03, 2022",11:25:00,11:30:00,G4,1,クイック・リボルブ,ハーピィの羽箒,混沌領域,龍の鏡,墓穴の指名者,ムーンナイト,0,,NaN
7,Lose,"March 05, 2022",NaN,NaN,G4,1,増殖するG,輝白竜ワイバースター,暗黒竜コラプサーペント,クイック・リボルブ,星遺物の守護竜,トライブリゲート,1,灰流うらら,NaN
8,Win,"March 05, 2022",NaN,NaN,G4,2,増殖するG,マグナヴァレット・ドラゴン,サイフレームギア・ドライバー,龍の鏡,増殖するG,シンクロン,1,増殖するG,NaN
9,Win,"March 05, 2022",02:02:00,02:22:00,G4,2,増殖するG,はるうらら,ヴァレット・トレーサー,ヴァレット・リチャージャー,ハーピィの羽根帚,オルフェゴール,0,,NaN


In [750]:
df_combine.to_csv('MasterDuel_1-50.csv')
df_combine.to_excel('MasterDuel_1-50.xlsx')